# **Homework 3 - Convolutional Neural Network**

This is the example code of homework 3 of the machine learning course by Prof. Hung-yi Lee.

In this homework, you are required to build a convolutional neural network for image classification, possibly with some advanced training tips.


There are three levels here:

**Easy**: Build a simple convolutional neural network as the baseline. (2 pts)

**Medium**: Design a better architecture or adopt different data augmentations to improve the performance. (2 pts)

**Hard**: Utilize provided unlabeled data to obtain better results. (2 pts)

## **About the Dataset**

The dataset used here is food-11, a collection of food images in 11 classes.

For the requirement in the homework, TAs slightly modified the data.
Please DO NOT access the original fully-labeled training data or testing labels.

Also, the modified dataset is for this course only, and any further distribution or commercial use is forbidden.

In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## **Import Packages**

First, we need to import packages that will be used later.

In this homework, we highly rely on **torchvision**, a library of PyTorch.

In [2]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder
from copy import deepcopy
import seaborn as sns

# This is for the progress bar.
from tqdm.notebook import tqdm

## **Dataset, Data Loader, and Transforms**

Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [3]:

tfm_norm = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# For test voting
train_tfm = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.7, 1.0), ratio=(.8, 1.2)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=.1, hue=0.07),
            transforms.ToTensor(),
            tfm_norm
        ])


test_tfm = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    tfm_norm
])


### Dataset of same images with different transforms

In [4]:
class RandomTransformDataset:
    def __init__(self, dataset, transform, random_time):
        self.dataset = dataset
        self.transform = transform
        self.random_time = random_time
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        jpg, _ = self.dataset[idx]
        imgs = [self.transform(jpg) for i in range(self.random_time)]
        return torch.stack(imgs, 0)
    
    @staticmethod
    def merge_batch(imgs):
        # (batch, RANDOM_NUM, 3, shape0, shape1)
        return imgs.reshape(-1, *imgs.shape[2:])
    
    def merge_predict(self, predicts):
        # (batch*RANOM_NUM, 11)
        res = []
        labels = torch.argmax(predicts, dim=1).squeeze()
        # (batch*RANOM_NUM)
        for prob in torch.split(labels, self.random_time):
            # (RANDOM_NUM)
            # voting
            res.append(torch.argmax(torch.bincount(prob)))
        # (batch)
        return  res



### OneHot Target Transform

In [5]:
onehot_lookup = torch.eye(11)
def onehot(label):
    return onehot_lookup[label]

In [6]:
batch_size = 64

# data_folder = "/data/ML2021/hw3/"
data_folder = ""

train_set = DatasetFolder(data_folder+"food-11/training/labeled", loader=lambda x: Image.open(x),extensions="jpg", transform=train_tfm, target_transform=onehot)
valid_set = DatasetFolder(data_folder+"food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm, target_transform=onehot)
# test_set = DatasetFolder(data_folder+"food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

unlabeled_set = DatasetFolder(data_folder+"food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
unlabeled_set.classes = train_set.classes
unlabeled_set.class_to_idx = train_set.class_to_idx

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
train_loader_noshuf = DataLoader(train_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)
unlab_loader = DataLoader(unlabeled_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
# test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [7]:
RANDOM_NUM = 8 # 8 different transform
test_set = DatasetFolder(data_folder+"food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=None)
rd_test_set = RandomTransformDataset(test_set, train_tfm, random_time=RANDOM_NUM)
rd_test_loader = DataLoader(rd_test_set, batch_size=batch_size//RANDOM_NUM*2, shuffle=False, num_workers=16)
test_loader = DataLoader(DatasetFolder(data_folder+"food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm), batch_size=batch_size, shuffle=False, num_workers=16)


## **Model**

The basic model here is simply a stack of convolutional layers followed by some fully-connected layers.

Since there are three channels for a color image (RGB), the input channels of the network must be three.
In each convolutional layer, typically the channels of inputs grow, while the height and width shrink (or remain unchanged, according to some hyperparameters like stride and padding).

Before fed into fully-connected layers, the feature map must be flattened into a single one-dimensional vector (for each image).
These features are then transformed by the fully-connected layers, and finally, we obtain the "logits" for each class.

### **WARNING -- You Must Know**
You are free to modify the model architecture here for further improvement.
However, if you want to use some well-known architectures such as ResNet50, please make sure **NOT** to load the pre-trained weights.
Using such pre-trained models is considered cheating and therefore you will be punished.
Similarly, it is your responsibility to make sure no pre-trained weights are used if you use **torch.hub** to load any modules.

For example, if you use ResNet-18 as your model:

model = torchvision.models.resnet18(pretrained=**False**) → This is fine.

model = torchvision.models.resnet18(pretrained=**True**)  → This is **NOT** allowed.

In [8]:
class BlockLinear(nn.Module):
    def __init__(self, ch_in, ch_out, act=None, use_bn=True, drop=0):
        super(BlockLinear, self).__init__()
        list = [
            nn.Linear(ch_in, ch_out)
        ]
        if use_bn: list.append(nn.BatchNorm1d(ch_out))
        if act: list.append(act)
        if drop > 0: list.append(nn.Dropout(drop))
        self.net = nn.Sequential(*list)

    def forward(self, x):
          return self.net(x)

In [10]:
class Classifier(nn.Module):
    def __init__(self, encoder, linears=[], drop=.3):
        super(Classifier, self).__init__()
        # input image size: [3, 224, 224]
        self.name = f'{encoder.__name__}_{"_".join([str(n) for n in linears])}'
        self.encoder = encoder(pretrained=False)
        self.dim = self.encoder.fc.in_features
        # self.encoder.fc = BlockLinear(512, 128, act=nn.ReLU(), use_bn=True)
        # self.n = [512, 128]
        self.n = [self.dim] + linears
        self.l = len(self.n)
        fc = [
            BlockLinear(self.n[i], self.n[i+1], act=nn.ReLU(), use_bn=True, drop=drop)
            for i in range(self.l-1)
        ]
        self.encoder.fc = nn.Sequential(*fc)
        self.out = nn.Linear(self.n[-1], 11)

    def forward(self, x):
        x = self.encoder(x)
        x = self.out(x)
        return x
    
    # Moveing Avg Update
    def ema_update(self, target, alpha):
        one_minus_alpha = 1. - alpha
        with torch.no_grad():
            for para, tar in zip(self.parameters(), target.parameters()):
                para.sub_(one_minus_alpha * (para - tar))
            
            for para, tar in zip(self.buffers(), target.buffers()):
                if para.dtype == torch.float:
                    para.sub_(one_minus_alpha * (para - tar))
                else:
                    para.copy_(tar)

## **Training**

You can finish supervised learning by simply running the provided code without any modification.

The function "get_pseudo_labels" is used for semi-supervised learning.
It is expected to get better performance if you use unlabeled data for semi-supervised learning.
However, you have to implement the function on your own and need to adjust several hyperparameters manually.

For more details about semi-supervised learning, please refer to [Prof. Lee's slides](https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/semi%20(v3).pdf).

Again, please notice that utilizing external data (or pre-trained model) for training is **prohibited**.

In [11]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(52728) # Waifu number
device = "cuda" if torch.cuda.is_available() else "cpu"

### Additionally define the psudolabels in Subset class

In [12]:
class SubsetCustomLabel:
    def __init__(self, dataset, labels, indices):
        self.dataset = dataset
        self.labels = labels
        self.indices = indices

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]][0], self.labels[idx]
    
    def set_transform(self, transform):
        self.dataset.transform = transform

### Custom log

In [13]:
class Log:
    def __init__(self, log_path='log', clear=True):
        self.log_path = log_path
        if clear: self.clean_file()
        self.log = open(log_path, 'a')
    
    def clean_file(self):
        if self.log_path in os.listdir():
            os.remove(self.log_path)
            print('remove log')
        
    def flush(self):
        self.log.flush()
    
    def write(self, log_str, is_print=True):
        self.log.write(log_str + "\n")
        if is_print: print(log_str)
    
    def write_epoch(self, total, cur, sections, is_write=True, is_print=True):
        log_str = f"[ {cur + 1:03d}/{total:03d} ]"
        for n, s in sections:
            log_str += f" {self.section(n, s)}"
        if is_write: self.write(log_str, is_print=is_print)
        else: return log_str
    
    def section(self, dest, dict):
        log_str = f"[ {dest:<5} |"
        flg = False
        for n, v in dict.items():
            if flg:
                log_str += ","
            log_str += f" {n} = {v:.5f}"
            flg = True
        log_str += " ]"
        return log_str
    

In [14]:
def get_pseudo_labels(model, dataloader, threshold=0.5, tempreture=3, is_labeled=False, labeld_alpha=.5):
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)
    idx = []
    targets = []
    if is_labeled:
        count = torch.zeros((11, 11), dtype=torch.float32)
    else:
        count = torch.zeros(11, dtype=torch.float32)
    # Iterate over the dataset by batches.
    c = 0
    for i, batch in tqdm(enumerate(dataloader), leave=False, desc='PseudoLabels'):
        img, ans = batch
        with torch.no_grad():
            logits = model(img.to(device))

        if is_labeled:
            l = ans.to(device).argmax(dim=-1)
            u = torch.unique(l).cpu()
            for i in u:
                count[i] += onehot_lookup[logits[l==i].argmax(dim=1).cpu()].sum(dim=0)

            targets.append(softmax(logits/tempreture).cpu()*(1-labeld_alpha) + ans*labeld_alpha)

        else:
            # Obtain the probability distributions by applying softmax on logits.
            probs = softmax(logits)
            # Filter the data and construct a new dataset.
            probs1 = probs.max(dim=1).values
            
            select = (probs1 > threshold)
            c = probs[select].sum(dim=0).cpu()
            count[:len(c)] += c
            if not (select.any() or is_labeled):
                continue
            
            probs_arg = probs[select].argmax(dim=1)
            targets.append(softmax(logits[select]/tempreture).cpu())
            idx += (torch.where(select)[0] + batch_size*i).tolist()


    # custom subset
    targets = torch.cat(targets, dim=0)
    if is_labeled:
        new = SubsetCustomLabel(dataloader.dataset, targets, torch.arange(len(targets)).tolist())
    else:
        new = SubsetCustomLabel(dataloader.dataset, targets, idx)
    model.train()
    return new, count

def get_semi_set(model, threshold=0.5, tempreture_l=1, tempreture_p=1, labeld_alpha=0.25):
    unlabeled_set.transform = tfm_weak
    train_set.transform = tfm_weak
    
    pseudo_set, count = get_pseudo_labels(
        model, unlab_loader, threshold=threshold, tempreture=tempreture_p)
    log.write(f"Use pseudo label: {len(pseudo_set)}")
    log.write("Pseudo Labels Distribution:\n" + 
        ", ".join([f'{i+1:02d}: {w*100:05.2f}' for i, w in enumerate(count/count.sum())]))
    
    labeled_set, confusion = get_pseudo_labels(
        model, train_loader_noshuf, tempreture=tempreture_l, is_labeled=True, labeld_alpha=labeld_alpha)
    

    labeled_loader = DataLoader(labeled_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True, drop_last=True)
    pseudo_loader = DataLoader(pseudo_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True, drop_last=True)
   
    unlabeled_set.transform = tfm_strong
    train_set.transform = tfm_strong
    return labeled_loader, pseudo_loader, confusion

In [15]:
def train(model, loader, criterion, shadow_model=None):
    model.train()

    train_loss = []
    train_accs = []
    for batch in tqdm(loader, desc='Train', leave=False):
        imgs, labels = batch
        logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()

        acc = (logits.argmax(dim=-1) == labels.to(device).argmax(dim=-1)).float().mean()

        train_loss.append(loss.item())
        train_accs.append(acc)

    if len(train_loss) == 0:
        train_loss = 0
        train_acc = 0
    else:
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)

    log_dict = ['Train', {'loss': train_loss, 'acc': train_acc}]
    if shadow_model:
        shadow_model.ema_update(model, ema_decay)
    return log_dict

def valid(model):    
    model.eval()
    
    valid_loss = []
    valid_accs = []
    confusion = torch.zeros((11, 11), dtype=float)
    for batch in tqdm(valid_loader, desc='Valid', leave=False):
        imgs, labels = batch
        imgs = imgs.to(device)
        with torch.no_grad():
            logits = model(imgs)

        loss = criterion(logits, labels.to(device))
        
        l = labels.to(device).argmax(dim=-1)
        acc = (logits.argmax(dim=-1) == l).float().mean()
        u = torch.unique(l)
        for i in u:
            confusion[i] += onehot_lookup[logits[l==i].argmax(dim=1).cpu()].sum(dim=0)

        valid_loss.append(loss.item())
        valid_accs.append(acc)

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)
    
    log_dict = ['Valid', {'loss': valid_loss, 'acc': valid_acc}]
    return log_dict, valid_acc, confusion

In [16]:
from torchvision.models import resnet18, resnet34, resnet50, resnext50_32x4d

tfm_weak = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.7, 1.0), ratio=(.8, 1.2)),
            transforms.RandomRotation(90),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=.15, hue=0.07),
            transforms.RandomGrayscale(p=0.1),
            transforms.ToTensor(),
            tfm_norm
        ])
tfm_strong = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.7, 1.0), ratio=(.8, 1.2)),
            transforms.RandomRotation(90),
            transforms.AutoAugment(),
            transforms.ToTensor(),
            transforms.RandomErasing(
                p=0.5, scale=(0.05, 0.2), ratio=(0.3, 2), value=0),
            tfm_norm
        ])

In [17]:
log = Log()
name_t = 'teacher'
name_s = 'student'
# model_ = [resnet34, [256, 64], 0.5]
model_ = [resnext50_32x4d, [1024, 128], 0.5]


remove log


In [18]:
torch.cuda.empty_cache()
model_path = "./model_{}.ckpt"
# for student
log_soft = nn.LogSoftmax(dim=1)
kl_div = nn.KLDivLoss(reduction='none', log_target=False)

# for teacher
kl_div_batch = nn.KLDivLoss(reduction='batchmean', log_target=False)
# criterion = lambda x, t: (kl_div(log_soft(x), t)*weight).sum(dim=1).mean()
criterion = lambda x, t: (kl_div_batch(log_soft(x), t))
# weight = torch.ones((1, 11)).float().to(device)

ema_decay = 0.9
epoch_t = 0
epoch_s = 5
round_s = 30

loader = train_loader
# ---------- Teacher ----------

log.write("="*10 + f" {name_t} " + "="*10)
log.flush()

# tfm_weak = train_tfm
unlabeled_set.transform = tfm_weak
train_set.transform = tfm_weak
model_t = Classifier(*model_).to(device)
# model_t.load_state_dict(torch.load(model_path.format(name_t)))

optimizer = torch.optim.AdamW(
    model_t.parameters(), lr=4e-4, weight_decay=1e-4
)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=epoch_t, eta_min=1e-6, verbose=True)

best_acc = 0
for epoch in range(epoch_t):
    # ---------- Training ----------
    train_dict = train(model_t, train_loader, criterion)
    # ---------- Validation ----------
    valid_dict, valid_acc, _ = valid(model_t)
    if valid_acc >= best_acc:
        best_acc = valid_acc
        best_model = model_t.state_dict()
        torch.save(model_t.state_dict(), model_path.format(name_t))
        log.write(f'Saving {name_t} model with acc {best_acc:.3f}')
    
    log.write_epoch(epoch_t, epoch, [train_dict, valid_dict])
    log.flush()
    lr_scheduler.step()

# ---------- Mutual Learning ----------

log.flush()
model_t.load_state_dict(torch.load(model_path.format(name_t)))
model_s = deepcopy(model_t) # clone teacher network

optimizer = torch.optim.AdamW(
    model_s.parameters(), lr=1e-4, weight_decay=5e-4)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=epoch_s*round_s, eta_min=0, verbose=True)
    
best_acc_t = best_acc
best_acc = 0
for round in range(round_s):
    log.write("="*10 + f" {name_s}_round {round+1:02d} " + "="*10)
    # ---------- Pseudo Labels ----------
    # Label datas by teacher

    valid_dict, valid_acc, confusion = valid(model_t)
    best_acc_t = max(valid_acc, best_acc_t)
    # valid_dict[0] = 'Teacher Valid'
    # log.write(log.section(*valid_dict))
    log.write("Confusion:\n" + ", ".join([f'{i+1:02d}: {w:.2f}' for i, w in enumerate(confusion.diag()/64)]))
    
    unlabeled_set.transform = tfm_weak
    train_set.transform = tfm_weak

    labeled_loader, pseudo_loader, confusion_tr = get_semi_set(
        model_t, threshold=.7, tempreture_l=1.5, tempreture_p=.7, labeld_alpha=0.25)

    count = confusion_tr.diag()
    count /= 280
    count = count*10

    weight_labeled = nn.functional.softmax(-count, dim=-1).reshape((1, 11)).to(device)*11
    weight_pseudo = nn.functional.softmax(count, dim=-1).reshape((1, 11)).to(device)*11

    log.write("Labeled Weight:\n" + ", ".join([f'{i+1:02d}: {w:04.2f}' for i, w in enumerate(weight_labeled[0])]))
    log.write("Pseudo Weight:\n" + ", ".join([f'{i+1:02d}: {w:04.2f}' for i, w in enumerate(weight_pseudo[0])]))

    criterion_labeled = lambda x, t: 1.5*(kl_div(log_soft(x), t)*weight_labeled).sum(dim=1).mean()
    criterion_pseudo = lambda x, t: 0.5*(kl_div(log_soft(x), t)*weight_pseudo).sum(dim=1).mean()

    # ---------- Student ----------
    
    torch.cuda.empty_cache()
    unlabeled_set.transform = tfm_strong
    train_set.transform = tfm_strong

    for epoch in range(epoch_s):
        # ---------- Training ----------
        # train_dict = train(model_s)
        secs = []
        if valid_acc >= 0.6:
            train_dict_p = train(model_s, pseudo_loader, criterion_pseudo, shadow_model=model_t)
            train_dict_p[0] = 'Train Pseudo'
            secs.append(train_dict_p)
        else:
            log.write("Abort Pseudo")
        train_dict_l = train(model_s, labeled_loader, criterion_labeled, shadow_model=model_t)
        train_dict_l[0] = 'Train Labeled'
        secs.append(train_dict_l)
        log.write_epoch(epoch_s, epoch, secs)

        # ---------- Validation ----------
        valid_dict, valid_acc, _ = valid(model_s)
        valid_dict_t, valid_acc_t, _ = valid(model_t)
        valid_dict_t[0] = 'Valid T'

        if valid_acc_t > best_acc_t:
        # if False:
            log.write(f'Saving {name_t} model with acc {valid_acc_t:.3f}')
            torch.save(model_t.state_dict(), model_path.format(name_t))
            best_acc_t = valid_acc_t

        if valid_acc >= best_acc:
            best_acc = valid_acc
            best_model = model_s.state_dict()
            torch.save(model_s.state_dict(), model_path.format(name_s))
            log.write(f'Saving {name_s} model with acc {best_acc:.3f}')
              
        log.write_epoch(epoch_s, epoch, [valid_dict, valid_dict_t])        
        log.flush()
        lr_scheduler.step()

========== teacher ==========
Adjusting learning rate of group 0 to 4.0000e-04.
Adjusting learning rate of group 0 to 1.0000e-04.
========== student_round 01 ==========


Valid:   0%|          | 0/11 [00:00<?, ?it/s]

Confusion:
01: 0.61, 02: 0.78, 03: 0.70, 04: 0.83, 05: 0.84, 06: 0.80, 07: 0.89, 08: 0.89, 09: 0.83, 10: 0.92, 11: 0.86


PseudoLabels: 0it [00:00, ?it/s]

Use pseudo label: 2232
Pseudo Labels Distribution:
01: 12.69, 02: 02.74, 03: 13.83, 04: 15.90, 05: 16.54, 06: 23.14, 07: 00.53, 08: 00.10, 09: 11.58, 10: 00.49, 11: 02.46


...

[ 005/005 ] [ Valid | loss = 1.03218, acc = 0.86307 ] [ Valid T | loss = 0.98280, acc = 0.86108 ]
Adjusting learning rate of group 0 to 7.0337e-05.
========== student_round 12 ==========


Valid:   0%|          | 0/11 [00:00<?, ?it/s]

Confusion:
01: 0.64, 02: 0.75, 03: 0.66, 04: 0.86, 05: 0.83, 06: 0.81, 07: 0.91, 08: 0.86, 09: 0.83, 10: 0.91, 11: 0.88


PseudoLabels: 0it [00:00, ?it/s]

RuntimeError: There were no tensor arguments to this function (e.g., you passed an empty list of Tensors), but no fallback function is registered for schema aten::_cat.  This usually means that this function requires a non-empty list of Tensors.  Available functions are [CPU, CUDA, QuantizedCPU, BackendSelect, Named, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradNestedTensor, UNKNOWN_TENSOR_TYPE_ID, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, Autocast, Batched, VmapMode].

CPU: registered at /pytorch/build/aten/src/ATen/RegisterCPU.cpp:5925 [kernel]
CUDA: registered at /pytorch/build/aten/src/ATen/RegisterCUDA.cpp:7100 [kernel]
QuantizedCPU: registered at /pytorch/build/aten/src/ATen/RegisterQuantizedCPU.cpp:641 [kernel]
BackendSelect: fallthrough registered at /pytorch/aten/src/ATen/core/BackendSelectFallbackKernel.cpp:3 [backend fallback]
Named: registered at /pytorch/aten/src/ATen/core/NamedRegistrations.cpp:7 [backend fallback]
AutogradOt

In [ ]:
torch.save(model_t.state_dict(), model_path.format(name_t+"_final"))

1. calculate weight based on confusion of valid
    - remove logit of fake labels?
    - argmax or confidence?
2. different weight on label and pseudo label depend on confidence
    - High confidence: less on labeled, more on pseudo label

In [19]:
torch.cuda.empty_cache()

## **Testing**

For inference, we need to make sure the model is in eval mode, and the order of the dataset should not be shuffled ("shuffle=False" in test_loader).

Last but not least, don't forget to save the predictions into a single CSV file.
The format of CSV file should follow the rules mentioned in the slides.

### **WARNING -- Keep in Mind**

Cheating includes but not limited to:
1.   using testing labels,
2.   submitting results to previous Kaggle competitions,
3.   sharing predictions with others,
4.   copying codes from any creatures on Earth,
5.   asking other people to do it for you.

Any violations bring you punishments from getting a discount on the final grade to failing the course.

It is your responsibility to check whether your code violates the rules.
When citing codes from the Internet, you should know what these codes exactly do.
You will **NOT** be tolerated if you break the rule and claim you don't know what these codes do.


In [18]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
model = Classifier(*model_).to(device)
model_path = f'./model_{name_t}.ckpt'
model.load_state_dict(torch.load(model_path))
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(rd_test_loader):
    imgs = rd_test_set.merge_batch(batch)

    with torch.no_grad():
        logits = model(imgs.to(device))
    
    # Take the class with greatest logit as prediction and record it.
    predictions.extend(rd_test_set.merge_predict(logits))

  0%|          | 0/210 [00:00<?, ?it/s]

In [19]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")

In [20]:
torch.cuda.empty_cache()